In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset Augmentation

This notebook has the purpose of applying some data augmetnation to the current dataset.

## Load Data

First we will read and process the files that contain the raw data. 

In [2]:
# Set and create the source and destination folders
videos_src_folder = '../data/raw/videos'
csvs_src_folder = '../data/raw/csvs'
dataset_dest_folder = '../data/processed/ImageDatasetRGB_Augmented'

try:
    if not os.path.exists(os.path.dirname(dataset_dest_folder)):
        os.mkdir(dataset_dest_folder)
        os.mkdir(os.path.join(dataset_dest_folder, 'features'))
        os.mkdir(os.path.join(dataset_dest_folder, 'labels'))    

except OSError as err:
    print(err)
    print('Dataset Destination Folders already exists')

We will delete any unrelated file.

In [3]:
# Read the content from the source folders
csvs = sorted(os.listdir(csvs_src_folder))
videos = sorted(os.listdir(videos_src_folder))

# Clean unecessary files
count_delated = 0
for i in range(len(csvs)):
    if csvs[i-count_delated][:2] == '._' :
        csvs = np.delete(csvs, i-count_delated)
        count_delated += 1
    elif 'gitkeep' in csvs[i-count_delated]:
        csvs = np.delete(csvs, i-count_delated)
        count_delated += 1
    elif 'DS_Store' in csvs[i-count_delated]:
        csvs = np.delete(csvs, i-count_delated)
        count_delated += 1

count_delated = 0
for i in range(len(videos)):
    if  videos[i-count_delated][:2] == '._' :
        videos = np.delete(videos, i-count_delated)
        count_delated += 1
    elif 'gitkeep' in videos[i-count_delated]:
        videos = np.delete(videos, i-count_delated)
        count_delated += 1
    elif 'DS_Store' in videos[i-count_delated]:
        videos = np.delete(videos, i-count_delated)
        count_delated += 1
        
csvs, videos

(array(['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', 'Animal61980.csv',
        'Animal62418.csv', 'HD_ChR2_480A.csv', 'HD_ChR2_510A.csv',
        'HD_ChR2_586A.csv', 'HD_YFP_037A.csv', 'HD_YFP_443A.csv',
        'HD_YFP_463A.csv', 'WT_ChR2_087A.csv', 'WT_ChR2_400A.csv',
        'WT_ChR2_425A.csv', 'WT_ChR2_635A.csv', 'WT_ChR2_654A.csv',
        'WT_YFP_154A.csv', 'WT_YFP_435A.csv', 'WT_YFP_535A.csv',
        'WT_YFP_602A.csv', 'WT_YFP_741A.csv', 'WT_YFP_792.csv'],
       dtype='<U16'),
 array(['037A.mp4', '087A.mp4', '1.mp4', '154A.mp4', '2.mp4', '3.mp4',
        '4.mp4', '400A.mp4', '425A.mp4', '435A.mp4', '443A.mp4',
        '463A.mp4', '480A.mp4', '5.mp4', '510A.mp4', '535A.mp4',
        '586A.mp4', '602A.mp4', '635A.mp4', '654A.mp4', '741A.mp4',
        '792.mp4', 'Video_Animal61980_10min.mp4',
        'Video_Animal62418_10min.mp4'], dtype='<U27'))

Lastly, we will sort them with the same order on both lists.

In [4]:
csvs = sorted(csvs)
videos_aux = []

for file in csvs:
    if 'Animal' in file:
        name = 'Video_' + file.split('.')[0] + '_10min.mp4'
    elif len(file.split('.')[0]) > 2:
        name = file.split('.')[0].split('_')[-1] + '.mp4'
    else: 
        name = file.split('.')[0] + '.mp4'
    videos_aux.append(videos[np.where(np.char.find(videos, name) == 0)][0])

videos = videos_aux
print(videos)
print(sorted(csvs))

['1.mp4', '2.mp4', '3.mp4', '4.mp4', '5.mp4', 'Video_Animal61980_10min.mp4', 'Video_Animal62418_10min.mp4', '480A.mp4', '510A.mp4', '586A.mp4', '037A.mp4', '443A.mp4', '463A.mp4', '087A.mp4', '400A.mp4', '425A.mp4', '635A.mp4', '654A.mp4', '154A.mp4', '435A.mp4', '535A.mp4', '602A.mp4', '741A.mp4', '792.mp4']
['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', 'Animal61980.csv', 'Animal62418.csv', 'HD_ChR2_480A.csv', 'HD_ChR2_510A.csv', 'HD_ChR2_586A.csv', 'HD_YFP_037A.csv', 'HD_YFP_443A.csv', 'HD_YFP_463A.csv', 'WT_ChR2_087A.csv', 'WT_ChR2_400A.csv', 'WT_ChR2_425A.csv', 'WT_ChR2_635A.csv', 'WT_ChR2_654A.csv', 'WT_YFP_154A.csv', 'WT_YFP_435A.csv', 'WT_YFP_535A.csv', 'WT_YFP_602A.csv', 'WT_YFP_741A.csv', 'WT_YFP_792.csv']


We will calsisfy videos regarding their recording conditions

## Process videos

We define a method to automatically find the central position of the mouse and crop the image.

In [6]:
def crop_image(image, expansion):
    
    # Start by converting the image to Gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Rescale the values to highlight constrasted areas
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Crop the box
    thresh2 = np.invert(thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    close = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    # We want to avoid noise, for that we will just consider the biggest countour since it is going to be the box
    c = 0
    c_len = len(cnts[0])
    for i in range(len(cnts[1:])):
        if len(cnts[i]) > c_len:
            c = i
            c_len = len(cnts[i])          
    
    # Obtain bounding rectangle to get the box coordinates
    x,y,w,h = cv2.boundingRect(cnts[c])

    # If the box is way to big, we won't need to crop it, else we will 
    # If the box isnot detected by the filter, then we can consider it is big enough to not crop it
    if not (x+w - x) < image.shape[0]//4:
        image = image[y:y+h, x:x+w]
        thresh = thresh[y:y+h, x:x+w]

    # We can now look for the mouse contour
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
   
    # We want to avoid noise, for that we will just consider the biggest countour since it is going to be the mouse, as long as it is not
    # to close to the extremes of the image, since that would mean it is an external object
    c = 0
    c_len = len(cnts[0])
    for i in range(len(cnts[1:])):
        if len(cnts[i]) > c_len:
            # Obtain bounding rectangle to get measurements
            x,y,w,h = cv2.boundingRect(cnts[i])
            if not (y < 100) or not (y+h > image.shape[0]-100):
                c = i
                c_len = len(cnts[i])

    # Crate a bounding box arround the mouse
    x,y,w,h = cv2.boundingRect(cnts[c])
    
    # Find centroid, this will be the center position of the mouse
    M = cv2.moments(cnts[c])            
    cX = int(M["m10"] / (M["m00"] + 1e-10))
    cY = int(M["m01"] / (M["m00"]+ 1e-10))
    midbody = [cX,cY]
    top = max(0, midbody[0] - expansion) - max(0, midbody[0] + expansion - image.shape[1])
    bottom = min(image.shape[1], midbody[0] + expansion) + max(0, expansion - midbody[0])
    left = max(0, midbody[1] - expansion) - max(0, midbody[1] + expansion - image.shape[0])
    right = min(image.shape[0], midbody[1] + expansion) + max(0, expansion - midbody[1])
   
    #Get cropped binary image and clean it with erosion
    #final = thresh[left:right,top:bottom]
    #kernel = np.ones((3, 3), np.uint8) 
    #final = cv2.erode(final, kernel, iterations=1) 
    #return final
    
    #Return cropped image
    return image[left:right,top:bottom]

Then, we can start processing the videos. We will generate up to 20 new videos with data augmentation.

In [115]:
expansion = 80
behaviours = ['grooming', 'rearing']
fps = 10
seen_videos= []

for _ in range(20):

    # Choose random video and csv
    while True:
        idx = np.random.randint(0,len(videos))
        if idx not in seen_videos:
            break
            
    csv = csvs[idx]
    video = videos[idx]
    seen_videos.append(idx)

    print(video, csv)

    # Read columns from csvs corresponding to the behaviours
    df = pd.read_csv(os.path.join(csvs_src_folder, csv), header=0)
    df.columns = map(str.lower, df.columns)
    
    if 'rearing' in df.columns:
        df = df[behaviours]
    else:
        rearing = np.maximum(df['rearing mig'], df['rearing paret'])
        df_aux = pd.DataFrame()
        df_aux['grooming'] = df['grooming']
        df_aux['rearing'] = rearing 
        df = df_aux
        
    # Change voids per 0 
    df.fillna(0, inplace=True)
    # Change data format
    df = df.astype(int)
    # Reset row indexes
    df.reset_index(inplace=True, drop=True)
    # Save df as csv
    df.to_csv(os.path.join(dataset_dest_folder, 'labels', csv.split('.')[0] + '_augmented.csv'), index=False)
    print(csv)
    print('DF: ', df.shape)

    # Get video frames
    os.mkdir(os.path.join(dataset_dest_folder, 'features',(csv.split('.')[0]+'_augmented')))
    vidcap = cv2.VideoCapture(os.path.join(videos_src_folder, video))              

    # Get the video FPS rate
    fps_in = vidcap.get(cv2.CAP_PROP_FPS)
    print('Video FPS: ', fps_in)

    # Start processing each frame
    success,image = vidcap.read()

    frames_in = 0
    frames_out = 0
    count = 0
    count_df = 0
    drop_indx = []

    # Define Image Data generator for data augmentation
    datagen = ImageDataGenerator(
        zoom_range = random.uniform(-0.5, 0.5),
        horizontal_flip = np.random.randint(0,1),
        brightness_range = (0.5, 1.5))
    # Define a seed
    seed = np.random.randint(1,100)

    while success:
        #If the video already has the standard FPS, we don't have to do anything 
        if  fps_in == fps:
            # Build frame name
            frame_name = 'frame'
            for i in range(4-len(str(count))):
                frame_name += '0'
            frame_name += str(count) + '.jpg'
    
            # Crop image so mouse is postioned in the center
            frame = crop_image(image, expansion)
            # Check correct format of frame
            if frame.shape != (160,160,3):
            #if frame.shape != (160,160):
                print('Error frame shape: ', frame.shape)

            # Perform random data augmentation
            for batch in datagen.flow(frame[None], batch_size=1, seed=seed):
                frame = np.array(batch[0], dtype=np.uint8)
                break
            
            # Save frame
            cv2.imwrite(os.path.join(dataset_dest_folder, 'features', (csv.split('.')[0] + '_augmented'), frame_name), frame)
            
            success, image = vidcap.read()
            count += 1
        # Else we will adjsut the frames we process so we get the standard FPS rate
        else:
            # We will caluculate the second where we are on the video and scale it to the desired FPS
            out_due = int(frames_in / fps_in * fps)
    
            if out_due > frames_out:
                frames_out += 1
                # Build frame name
                frame_name = 'frame'
                for i in range(4-len(str(count))):
                    frame_name += '0'
                frame_name += str(count) + '.jpg'
        
                # Crop image so mouse is postioned in the center
                frame = crop_image(image, expansion)
                # Check correct format of frame
                if frame.shape != (160,160,3):
                #if frame.shape != (160,160):
                    print('Error frame shape: ', frame.shape)
                    
                # Perform random data augmentation
                for batch in datagen.flow(frame[None], batch_size=1, seed=seed):
                    frame = np.array(batch[0], dtype=np.uint8)
                    break
                    
                # Save frame
                cv2.imwrite(os.path.join(dataset_dest_folder, 'features', (csv.split('.')[0] + '_augmented'), frame_name), frame) 
                count += 1  
            # We will remove those rows that don't correspond to any frame
            else:
                if count_df < len(df):
                    drop_indx.append(count_df)
                    
            frames_in += 1
            count_df += 1
            success, image = vidcap.read()
            
    if  fps_in != fps:
        df = df.drop(drop_indx, axis=0)
        # Save df as csv
        df.to_csv(os.path.join(dataset_dest_folder, 'labels', csv.split('.')[0] + '_augmented.csv'), index=False)
        
    print("Label rows: ", df.shape)
    print("Video frames: ", count)

2.mp4 2.csv
2.csv
DF:  (12055, 2)
Video FPS:  29.97
Label rows:  (4022, 2)
Video frames:  4022
635A.mp4 WT_ChR2_635A.csv
WT_ChR2_635A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
087A.mp4 WT_ChR2_087A.csv
WT_ChR2_087A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
037A.mp4 HD_YFP_037A.csv
HD_YFP_037A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
480A.mp4 HD_ChR2_480A.csv
HD_ChR2_480A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
510A.mp4 HD_ChR2_510A.csv
HD_ChR2_510A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
654A.mp4 WT_ChR2_654A.csv
WT_ChR2_654A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
443A.mp4 HD_YFP_443A.csv
HD_YFP_443A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows:  (6600, 2)
Video frames:  6600
154A.mp4 WT_YFP_154A.csv
WT_YFP_154A.csv
DF:  (6600, 2)
Video FPS:  10.0
Label rows

Lastly we will create an split file, where all folder will correspond to the taining set. 

In [118]:
dataset_path = '../data/processed/ImageDatasetRGB_Augmented/'

features = sorted(os.listdir(os.path.join(dataset_path, 'features')))
if '.DS_Store' in features:
    features.remove('.DS_Store')

set_list = []

for i in features:
    set_list.append('train')

print(set_list)

['train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train']


In [119]:
dataset_path = '../data/processed/'
dataset_split = pd.DataFrame({
    'name': features,
    'set': set_list
})
dataset_split.to_csv(os.path.join(dataset_path, 'augmented_split.csv'), index=None)